## Introduction & Understanding The Data

Whether we realize it or not, without steel our modern society would be very different! Infrastructure built with steel is resilient to heavy stress from human and nature, and therefore the production of steel is an important process. This competition asks us to help make the production process of steel more efficient by identifying defects. 

In this notebook I will implement a CNN architecture (ResUNet), and see if we can identify these defects! The original paper that proposes this architecture is [ResUNet-a: a deep learning framework for semantic segmentation of remotely sensed data](https://arxiv.org/abs/1904.00592).

If you like my work please upvote me. Upvoting is a great postive feedback, and it helps people like me know that the work was worth it. I did this one rather quickly, so if I made a mistake, please leave a comment in the comments section to help me out. Many thanks in advance!

In [11]:
# some basic imports
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

# import imgaug as aug
# import imgaug as ia
# from imgaug import augmenters as iaa


This competition is about instance segmentation. That is there are multiple instance of interest in the image and we need to identify each one of them. In the train file, the first column has the image id and class id seperated by a underscore. For example, in the cell below the image 0002cc93b.jpg has a fault class 1 and the mask for this fault is given. There are four fault classes and so each image appears four times in the train.csv dataset. 

In [3]:
import os
# os.listdir('../input/severstal-steel-defect-detection/')
from os.path import isfile, join

files = [f for f in os.listdir('images/') if isfile(join('images/',f))]

In [4]:
print(files)

['0a9aaba9a.jpg', '0a5a82b86.jpg', '0a26aceb2.jpg', '0a8fddf7a.jpg', '0a5cff3a7.jpg', '0a3bbea4d.jpg', '0a29ef6f9.jpg', '0a5ad2f0f.jpg', '000a4bcdd.jpg', '0a4ef8ee7.jpg', '0a1cade03.jpg', '0a8cb8ddf.jpg', '0a21ce787.jpg', '0a4ad45a5.jpg', '0a9a7a6c7.jpg']


In [5]:
# path to the train images
# img_path = '../input/severstal-steel-defect-detection/train_images/'
img_path = 'images/'

In [6]:
pd.read_csv('train.csv').head()

,ImageId_ClassId,EncodedPixels
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0002cc93b.jpg_2,NaN
2,0002cc93b.jpg_3,NaN
3,0002cc93b.jpg_4,NaN
4,00031f466.jpg_1,NaN


In [7]:
# load full data and label no mask as -1

train_df = pd.read_csv('train.csv').fillna(-1)

In [27]:
# ImageId and ClassId as two seperate columns

train_df['ImageId'] = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
train_df['ClassId'] = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[1])
train_df['enhance'] = 0

# dict with ClassId and EncodedPixels and group all faults by ImageId

train_df['ClassId_EncodedPixels'] = train_df.apply(lambda row: (row['ClassId'], row['EncodedPixels']), axis = 1)
grouped_EncodedPixels = train_df.groupby('ImageId')['ClassId_EncodedPixels'].apply(list)


In [29]:
train_df.head()

,ImageId_ClassId,EncodedPixels,ImageId,ClassId,ClassId_EncodedPixels,enhance
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...,0002cc93b.jpg,1,"(1, 29102 12 29346 24 29602 24 29858 24 30114 ...",0
1,0002cc93b.jpg_2,-1,0002cc93b.jpg,2,"(2, -1)",0
2,0002cc93b.jpg_3,-1,0002cc93b.jpg,3,"(3, -1)",0
3,0002cc93b.jpg_4,-1,0002cc93b.jpg,4,"(4, -1)",0
4,00031f466.jpg_1,-1,00031f466.jpg,1,"(1, -1)",0


In [28]:
df_grouped_EncodedPixels = pd.DataFrame(grouped_EncodedPixels)
df_grouped_EncodedPixels.head()

,ClassId_EncodedPixels
ImageId,
0002cc93b.jpg,"[(1, 29102 12 29346 24 29602 24 29858 24 30114..."
00031f466.jpg,"[(1, -1), (2, -1), (3, -1), (4, -1)]"
000418bfc.jpg,"[(1, -1), (2, -1), (3, -1), (4, -1)]"
000789191.jpg,"[(1, -1), (2, -1), (3, -1), (4, -1)]"
0007a71bf.jpg,"[(1, -1), (2, -1), (3, 18661 28 18863 82 19091..."


## Data distribution
The dataset has lots of non error images while errors are 14,2 % of the images, and within the 7095 errors, we have a big amount of class 2 errors (72,5 %)
The data sample is clearly umbalanced

In [25]:
# for each fault class, here the mumber of positive incidences
count_classId = [0,0,0,0,0]
for i, row in tqdm(train_df.iterrows()):
    if row['EncodedPixels'] != -1:
        count_classId[int(row['ClassId'])] += 1
    else:
        count_classId[0] += 1

print('\nFirst iter')

# represents number of samples with 0, 1, 2, 3 or 4 fault classes simultaneously
defaults_by_classId = [0,0,0,0,0]
unique_defaults = [0,0,0,0]
unique_default_id = {}
for i, row in tqdm(df_grouped_EncodedPixels.iterrows()):
    n = 0
    for xx in row:
        for x in xx:
            if x[1] != -1:
                n += 1
                v = int(x[0])
        if n == 1:
            unique_defaults[v-1] += 1
            unique_default_id[i] = v
            
    defaults_by_classId[n] +=1
    
print('\nSecond iter')        
num_defaults = sum(count_classId) - count_classId[0]

print('Num of elements by class:{} \
        \nNum of elements: {} \
        \nNum defaults: {} \
        \nNum of images: {} \
        \ntrain_df length: {} \
        \nDefaults by Class Id: {} \
        \nUnique defaults by class: {}'.format(count_classId, 
                                    sum(count_classId),
                                    num_defaults,
                                    len(grouped_EncodedPixels),
                                    len(train_df),
                                    defaults_by_classId,
                                    unique_defaults))

50272it [00:03, 12822.27it/s]
1254it [00:00, 12539.35it/s]


First iter


12568it [00:00, 13277.38it/s]


Second iter
Num of elements by class:[43177, 897, 247, 5150, 801]         
Num of elements: 50272         
Num defaults: 7095         
Num of images: 12568         
train_df length: 50272         
Defaults by Class Id: [5902, 6239, 425, 2, 0]         
Unique defaults by class: [769, 195, 4759, 516]


In [26]:
print(len(unique_default_id))

6239


We'll create a new dataset reusing the undersampled classes so to have a complete representation of all of them.
we'll apply some random data augmentation, applying shifts up and down, resizing, and rolling with different values. 
With this we'll be able to obtain someting like 10x number of samples for the most underepresented ones.

In [30]:
# Create deep copy of train_df dataframe where I'll increase cases
# in order to homogeinize the training set.
train_df_enh = train_df.copy(deep=True)

In [ ]:
for i in unique_default_id.keys():
    num_enh = random.randit(1,5)
    for n in range(num_enh):
        # have to include new lines with some kind of modification of the 
        # image and mask
        train_df_enh.append()

In [ ]:
# picking up 10 examples with at least two faults for visualization

samples = []
for f in files:
    r = grouped_EncodedPixels[f]
    if (len([x[1] for x in r if x[1] != -1]) >= 1) and (len(samples) < 10):
        samples.append(f)

In [ ]:
samples

In [ ]:
# Masks are obviously encoded. we use the following function to 
# decode the masks. I picked this function up from a Kaggle kernel, 
# so thanks to the authors of https://www.kaggle.com/robertkag/rle-to-mask-converter

def rleToMask(rleString, height, width):
    rows, cols = height, width
    rleNumbers = [int(numstring) for numstring in rleString.split(' ')]
    rlePairs = np.array(rleNumbers).reshape(-1,2)
    img = np.zeros(rows * cols, dtype=np.uint8)
    for index, length in rlePairs:
        index -= 1
        img[index : index+length] = 255
    img = img.reshape(cols,rows)
    img = img.T
    return img

In [ ]:
# show steel image with four classes of faults in seperate columns
def show_image_masks(img, masks, convert_to_float=False):
    fig, ax = plt.subplots(nrows=1, ncols=4, sharey=True, figsize=(20,10))
    cmaps = ["Reds", "Blues", "Greens", "Purples"]
    for i, mask in enumerate(masks):
        if mask == -1:
            mask_decoded = np.zeros((256, 1600))
        else:
            mask_decoded = rleToMask(mask, 256, 1600)
        ax[i].imshow(img)
        ax[i].imshow(mask_decoded, alpha=0.5, cmap=cmaps[i])

In [ ]:
# visualize the image we picked up earlier with mask
for sample in samples:
    mask_1, mask_2, mask_3, mask_4 = grouped_EncodedPixels[sample]
    masks = [mask_1[1], mask_2[1], mask_3[1], mask_4[1]]
    img = cv2.imread(os.path.join(img_path, sample))
    show_image_masks(img, masks)

## Network Configurations

As mentioned earlier, this notebook I will implement a ResUNet to predict the different classes of faults. As the image we have is colorful, we will have 3 channels in our input layer. As we have four types of masks, we will have four channels in output layer to pick up different categories of faults.

In [ ]:
# network configuration parameters
# original image is 1600x256, so we will resize it to 1/4 (400,64) to test model

img_w = 400
img_h = 64
img_size = (img_w, img_h)

# batch size for training unet

batch_size = 16

# kernel size 3x3

k_size = 3 

# split of training and validation set

val_size = .20

In [ ]:
# imports for building the network
import tensorflow as tf

from tensorflow import reduce_sum
from tensorflow.keras.backend import pow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, UpSampling2D, Concatenate, Add, Flatten
from tensorflow.keras.losses import binary_crossentropy

from sklearn.model_selection import train_test_split

# Data Augmentation
## Forked from: https://github.com/aleju/imgaug

In [ ]:

sometimes = lambda aug: iaa.Sometimes(0.5, aug)
seq = iaa.Sequential(
        [
            # apply the following augmenters to most images
            iaa.Fliplr(0.5), # horizontally flip 50% of all images
            iaa.Flipud(0.2), # vertically flip 20% of all images
            sometimes(iaa.Affine(
                # scale images to 90-110% of their size, individually per axis
                scale={"x": (0.9, 1.1), "y": (0.9, 1.1)}, 
                # traslate by -10 to +10 percent
                translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, 
                # use nearest neighbour or bilinear interpolation (fast)
                order=[0, 1], 
                # if mode is constant, use a cval between 0 and 255
                cval=(0, 255), 
                # use any of scikit-image's warping modes
                mode=ia.ALL
            ))
        ],
        random_order=True)

## Data Generator

To push the data to our model, we will create a custom data generator. A generator lets us load data progressively, instead of loading it all into memory at once. A custom generator allows us to also fit in more customization during the time of loading the data. As the model is being procssed in the GPU, we can use a custom generator to pre-process images via a generator. At this time, we can also take advantage multiple processors to parallelize our pre-processing.

In [ ]:
# new DataGenerator class
# improved with a train and a test position 
# uses image transform function to process te image

class DataGenerator(Sequence):

    def __init__(self, 
                 image_filenames, 
                 labels,
                 image_size = (400,64),
                 batch_size=16, 
                 is_train=True,
                 mix=False, 
                 augment=False):
        
        self.image_filenames, self.labels = image_filenames, labels
        self.image_size = image_size
        self.batch_size = batch_size
        self.is_train = is_train
        self.is_augment = augment
        if(self.is_train):
            self.on_epoch_end()
        self.is_mix = mix

    def __len__(self):
        # number of batches per epoch
        return int(np.ceil(len(self.image_filenames) / float(self.batch_size)))

    def __getitem__(self, idx):
        # generate one batch of data
        batch_x = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        if(self.is_train):
            return self.train_generate(batch_x, batch_y)
        return self.valid_generate(batch_x, batch_y)

    def on_epoch_end(self):
        if(self.is_train):
            self.image_filenames, self.labels = shuffle(self.image_filenames, self.labels)
        else:
            pass
    
    def image_transform(self, image):
        resized = cv2.resize(image, self.img_size , interpolation=cv2.INTER_CUBIC)
        return resized

    def mix_up(self, x, y):
        lam = np.random.beta(0.2, 0.4)
        ori_index = np.arange(int(len(x)))
        index_array = np.arange(int(len(x)))
        np.random.shuffle(index_array)        
        
        mixed_x = lam * x[ori_index] + (1 - lam) * x[index_array]
        mixed_y = lam * y[ori_index] + (1 - lam) * y[index_array]
        
        return mixed_x, mixed_y

    def train_generate(self, batch_x, batch_y):
        batch_images = []
        for (i, label) in zip(batch_x, batch_y):
            # file=f"/mnt/DATA-SSD/DataSandbox/APTOS/train_images/{i}.png"
            file = f"images/{i}"
            img = cv2.imread(file)
            # any image modification needed
            img = img_transform(img,2, size)
            if(self.is_augment):
                img = seq.augment_image(img)
            batch_images.append(img)
            
        batch_images = np.array(batch_images, np.float32) / 255
        batch_y = np.array(batch_y, np.float32)
        if(self.is_mix):
            batch_images, batch_y = self.mix_up(batch_images, batch_y)
        return batch_images, batch_y

    def valid_generate(self, batch_x, batch_y):
        batch_images = []
        for (i, label) in zip(batch_x, batch_y):
            # file=f"/mnt/DATA-SSD/DataSandbox/APTOS/train_images/{i}.png"
            file = f"images/{i}"
            img = cv2.imread(file)
            # any image modification needed
            img = img_transform(img,2,size)
            batch_images.append(img)
        batch_images = np.array(batch_images, np.float32) / 255
        batch_y = np.array(batch_y, np.float32)
        return batch_images, batch_y


In [ ]:
# create a dict of all the masks
masks = {}
for index, row in train_df[train_df['EncodedPixels']!=-1].iterrows():
    masks[row['ImageId_ClassId']] = row['EncodedPixels']

In [ ]:
# split the training data into train and validation set (stratified)
X_train, X_val = train_test_split(train_df[train_df['EncodedPixels']!=-1]['ImageId'].unique(), test_size=val_size, random_state=42)

In [ ]:
params = {'img_h': img_h,
          'img_w': img_w,
          'batch_size': batch_size,
          'shuffle': True}

# Get Generators
training_generator = DataGenerator(X_train, masks, **params)
validation_generator = DataGenerator(X_val, masks, **params)

In [ ]:
# check out the shapes
x, y = training_generator.__getitem__(0)
print(x.shape, y.shape)

In [ ]:
# lets visualize some images with their faults to make sure our data generator is working like it should
for ix in range(0,batch_size):
    if y[ix].sum() > 0:
        img = x[ix]
        masks_temp = [y[ix][...,i] for i in range(0,4)]
        show_image_masks(img, masks_temp)

## Resunet

In this section we will define the building blocks for our network and train our network.

In [ ]:
def bn_act(x, act=True):
    # batch normalization layer with a relu activation layer
    x = tf.keras.layers.BatchNormalization()(x)
    if act == True:
        x = tf.keras.layers.Activation('relu')(x)
    return x

In [ ]:
def conv_block(x, filters, kernel_size=3, padding='same', strides=1):
    # convolutional layer which always uses the batch normalization layer
    conv = bn_act(x)
    conv = Conv2D(filters, kernel_size, padding=padding, strides=strides)(conv)
    return conv

In [ ]:
def stem(x, filters, kernel_size=3, padding='same', strides=1):
    conv = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    conv = conv_block(conv, filters, kernel_size, padding, strides)
    shortcut = Conv2D(filters, kernel_size=1, padding=padding, strides=strides)(x)
    shortcut = bn_act(shortcut, act=False)
    output = Add()([conv, shortcut])
    return output

In [ ]:
def residual_block(x, filters, kernel_size=3, padding='same', strides=1):
    res = conv_block(x, filters, k_size, padding, strides)
    res = conv_block(res, filters, k_size, padding, 1)
    shortcut = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    shortcut = bn_act(shortcut, act=False)
    output = Add()([shortcut, res])
    return output

In [ ]:
def upsample_concat_block(x, xskip):
    u = UpSampling2D((2,2))(x)
    c = Concatenate()([u, xskip])
    return c

In [ ]:
def ResUNet(img_h, img_w):
    f = [16, 32, 64, 128, 256]
    inputs = Input((img_h, img_w, 3))
    
    ## Encoder
    e0 = inputs
    e1 = stem(e0, f[0])
    e2 = residual_block(e1, f[1], strides=2)
    e3 = residual_block(e2, f[2], strides=2)
    e4 = residual_block(e3, f[3], strides=2)
    e5 = residual_block(e4, f[4], strides=2)
    
    ## Bridge
    b0 = conv_block(e5, f[4], strides=1)
    b1 = conv_block(b0, f[4], strides=1)
    
    ## Decoder
    u1 = upsample_concat_block(b1, e4)
    d1 = residual_block(u1, f[4])
    
    u2 = upsample_concat_block(d1, e3)
    d2 = residual_block(u2, f[3])
    
    u3 = upsample_concat_block(d2, e2)
    d3 = residual_block(u3, f[2])
    
    u4 = upsample_concat_block(d3, e1)
    d4 = residual_block(u4, f[1])
    
    outputs = tf.keras.layers.Conv2D(4, (1, 1), padding="same", activation="sigmoid")(d4)
    model = tf.keras.models.Model(inputs, outputs)
    return model

## Loss Functions

As our classes are highly imbalanced and we are stacking four output layers at once, it is even more important to get the loss function right. Here I will aggregate important loss functions so you can reuse and experiment with them along with me.

In [ ]:
# Dice similarity coefficient loss
# https://github.com/nabsabraham/focal-tversky-unet

def dsc(y_true, y_pred):
    smooth = 1.
    y_true_f = Flatten()(y_true)
    y_pred_f = Flatten()(y_pred)
    intersection = reduce_sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (reduce_sum(y_true_f) + reduce_sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dsc(y_true, y_pred)
    return loss

def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

In [ ]:
# Focal Tversky loss
# https://github.com/nabsabraham/focal-tversky-unet

def tversky(y_true, y_pred, smooth=1e-6):
    y_true_pos = tf.keras.layers.Flatten()(y_true)
    y_pred_pos = tf.keras.layers.Flatten()(y_pred)
    true_pos = tf.reduce_sum(y_true_pos * y_pred_pos)
    false_neg = tf.reduce_sum(y_true_pos * (1-y_pred_pos))
    false_pos = tf.reduce_sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.7
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)

def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true,y_pred)

def focal_tversky_loss(y_true,y_pred):
    pt_1 = tversky(y_true, y_pred)
    gamma = 0.75
    return tf.keras.backend.pow((1-pt_1), gamma)

## Compile & Fit The Model

In [ ]:
model = ResUNet(img_h=img_h, img_w=img_w)
adam = tf.keras.optimizers.Adam(lr = 0.05, epsilon = 0.1)
model.compile(optimizer=adam, loss=focal_tversky_loss, metrics=[dsc, tversky])

In [ ]:
try:
    model.load_weights('../input/severstal-data/ResUNetSteelFaultOnly.h5')
except OSError:
    print('You need to run the model and load the trained model')

In [ ]:
history = model.fit_generator(generator=training_generator, validation_data=validation_generator, epochs=5, verbose=1)

In [ ]:
model.save('./ResUNetSteelFaultOnly.h5')

## Model Insights

In this section we take a look at the performance of our model and visually inspect how our predictions look like.

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.figure(figsize=(20,5))
plt.subplot(1,2,1)
plt.plot(history.history['tversky'])
plt.plot(history.history['val_tversky'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

# summarize history for loss
plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

In [ ]:
# a function to plot image with mask and image with predicted mask next to each other
def viz_single_fault(img, mask, pred, image_class):
    
    fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(15,5))
    
    cmaps = ["Reds", "Blues", "Greens", "Purples"]
    
    ax[0].imshow(img)
    ax[0].imshow(mask, alpha=0.3, cmap=cmaps[image_class-1])
    ax[0].set_title('Mask - Defect Class %s' % image_class)
    
    ax[1].imshow(img)
    ax[1].imshow(pred, alpha=0.3, cmap=cmaps[image_class-1])
    ax[1].set_title('Predicted Mask - Defect Class %s' % image_class)
    
    plt.show()

To get a better understanding of our model we will calculate the IoU score. If you are unfamiliar with the concept of IoU, you can read more of it here: http://ronny.rest/tutorials/module/localization_001/iou/.

In [ ]:
# https://www.jeremyjordan.me/evaluating-image-segmentation-models/
def calculate_iou(target, prediction):
    intersection = np.logical_and(target, prediction)
    union = np.logical_or(target, prediction)
    if np.sum(union) == 0:
        iou_score = 0
    else:
        iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

In [ ]:
# lets loop over the predictions and print 5 of each image cases with defects
count = 0
# a list to keep count of the number of plots made per image class
class_viz_count = [0,0,0,0]
# to keep the total iou score per image class
class_iou_score = [0, 0, 0, 0]
# to keep sum of mask pixels per image class
class_mask_sum = [0, 0, 0, 0]
# to keep sum of predicted mask pixels per image class
class_pred_sum = [0, 0, 0, 0]

# loop over to all the batches in one epoch 
for i in range(0, validation_generator.__len__()):
    # get a batch of image, true mask, and predicted mask
    x, y = validation_generator.__getitem__(i)
    predictions = model.predict(x)
    
    # loop through x to get all the images in the batch
    for idx, val in enumerate(x):
        # we are only interested if there is a fault. if we are dropping images with no faults before this will become redundant
        if y[idx].sum() > 0: 
            # get an image and convert to make it matplotlib.pyplot friendly
            img = x[idx]
            img = cv2.cvtColor(img.astype('float32'), cv2.COLOR_BGR2RGB)
            # loop over the four ourput layers to create a list of all the masks for this image
            masks_temp = [y[idx][...,i] for i in range(0,4)]
            # loop over the four output layers to create a list of all the predictions for this image
            preds_temp = [predictions[idx][...,i] for i in range(0,4)]
            # turn to binary (prediction) mask 
            preds_temp = [p > .5 for p in preds_temp]
            
            for i, (mask, pred) in enumerate(zip(masks_temp, preds_temp)):
                image_class = i + 1
                class_iou_score[i] += calculate_iou(mask, pred)
                class_mask_sum[i] += mask.sum()
                class_pred_sum[i] += pred.sum()
                if mask.sum() > 0 and class_viz_count[i] < 5:
                    viz_single_fault(img, mask, pred, image_class)
                    class_viz_count[i] += 1

In [ ]:
# visualize the IoU, Sum of pixel for mask given, and sum of pixel for the prediction.
class_ids = [1,2,3,4]
plt.figure(figsize=(20,5))
plt.subplot(1,3,1)
y_pos = np.arange(len(class_ids))
plt.bar(y_pos, class_iou_score)
plt.xticks(y_pos, class_ids)
plt.title('IoU score per class')
plt.ylabel('IoU Sum')
plt.xlabel('class id')
plt.subplot(1,3,2)
plt.bar(y_pos, class_mask_sum)
plt.xticks(y_pos, class_ids)
plt.title('labeled mask pixel sum per class')
plt.ylabel('pixel sum')
plt.xlabel('class id')
plt.ticklabel_format(axis='y',style='sci',scilimits=(1,4))
plt.subplot(1,3,3)
plt.bar(y_pos, class_pred_sum)
plt.xticks(y_pos, class_ids)
plt.title('predicted mask pixel sum per class')
plt.ylabel(' pixel sum')
plt.xlabel('class id')
plt.ticklabel_format(axis='y',style='sci',scilimits=(1,4))
plt.show()

Its obviously and unfortunately clear that we don't see enough sample of the other classes so we don't make a any prediction on them. I do wish we made some prediciton on class 4 but oh well, life building AI isn't easy =( 

## Making Predictions & Submission File

Now that our model is trained lets make a submission and file it in!

In [ ]:
# return tensor in the right shape for prediction 
def get_test_tensor(img_dir, img_h, img_w, channels=3):

    X = np.empty((1, img_h, img_w, channels))

    # Store sample
    image = cv2.imread(img_dir)
    image_resized = cv2.resize(image, (img_w, img_h))
    image_resized = np.array(image_resized, dtype=np.float64)
    X[0,] = image_resized

    return X

In [ ]:
# this is an awesome little function to remove small spots in our predictions

from skimage import morphology

def remove_small_regions(img, size):
    """Morphologically removes small (less than size) connected regions of 0s or 1s."""
    img = morphology.remove_small_objects(img, size)
    img = morphology.remove_small_holes(img, size)
    return img

In [ ]:
import glob
# get all files using glob
test_files = [f for f in glob.glob('../input/severstal-steel-defect-detection/test_images/' + "*.jpg", recursive=True)]

In [ ]:
# we will use this function to convert our mask into RLE. Thanks to the authors of: https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
submission = []

# a function to apply all the processing steps necessery to each of the individual masks
def process_pred_mask(pred_mask):
    
    pred_mask = cv2.resize(pred_mask.astype('float32'),(1600, 256))
    pred_mask = (pred_mask > .5).astype(int)
    pred_mask = remove_small_regions(pred_mask, 0.02 * np.prod(512)) * 255
    pred_mask = mask2rle(pred_mask)
    
    #if len(pred_mask) < 1:
    #    pred_mask = ''
    
    return pred_mask

# loop over all the test images
for f in test_files:
    # get test tensor, output is in shape: (1, 256, 512, 3)
    test = get_test_tensor(f, img_h, img_w) 
    # get prediction, output is in shape: (1, 256, 512, 4)
    pred_masks = model.predict(test) 
    # get a list of masks with shape: 256, 512
    pred_masks = [pred_masks[0][...,i] for i in range(0,4)]
    # apply all the processing steps to each of the mask
    pred_masks = [process_pred_mask(pred_mask) for pred_mask in pred_masks]
    # get our image id
    id = f.split('/')[-1]
    # create ImageId_ClassId and get the EncodedPixels for the class ID, and append to our submissions list
    [submission.append((id+'_%s' % (k+1), pred_mask)) for k, pred_mask in enumerate(pred_masks)]

In [ ]:
# convert to a csv
submission_df = pd.DataFrame(submission, columns=['ImageId_ClassId', 'EncodedPixels'])
# check out some predictions and see if it looks good
submission_df[ submission_df['EncodedPixels'] != '1 1'].head()

In [ ]:
# write it out
submission_df.to_csv('./submission.csv', index=False)

## Summary Section

1. For this competition, as it is a Kernel only competition, your maximum Kernel runtime is 1 hour. I forgot that like a dummy and in my first attempt, this version of the [kernel](https://www.kaggle.com/ekhtiar/eda-resunet-tensorflow-keras?scriptVersionId=18015343), I didn't manage to make a submission. In the next iteration, I will save the model and re-run it.
2. After pre-training our model for 50 epoch, I managed to load that and make a submission in this version of the [kernel](https://www.kaggle.com/ekhtiar/eda-resunet-tensorflow-keras?scriptVersionId=18052749). In this kernel we just had 5 additional epochs on top of the pre-trained model and the score was 0.71469.
3. Its very obvious that for Image class 3 we are doing a very good job. For the rest of the classes, we are not even being triggered. This is due to the imbalance in our classes. I tried dropping all the images with no masks at all, and retraining the model in this [kernel](https://www.kaggle.com/ekhtiar/resunet-baseline-with-tensorflow?scriptVersionId=18077319). However, it didn't help!
4. From all the kernels, we can observe that 20 epoch is good enough. So I will reduce my epochs to 20 from now on.
5. The loss function was changed to focal tversky loss, in this [kernel](https://www.kaggle.com/ekhtiar/resunet-baseline-on-tensorflow-w-tutorial-notes?scriptVersionId=18088542). We still didn't improve our predictions for other classes. One thing for sure, we need to zoom into the performance of the model more. And therefore section "Model Insights" is created/extended.
6. In future I will take step to balance the data more for each of the classes during the training process.

Okay guys! This is just a start and hopefully a good start. I will be back soon with update to make this much more professional. 

If you like my work please upvote me. Upvoting is a great postive feedback, and it helps people like me know that the work was worth it. I did this one rather quickly, so if I made a mistake, please leave a comment in the comments section to help me out. Many thanks in advance!